$Input: (B, C_{in}, H_{in}, W_{in})$ 

$Output: (B, C_{out}, H_{out}, W_{out})$

$$H_{out} = floor\left( \frac{H_{in} + 2\times pad[0] - dilation[0] \times (kernel - 1) -1 }{stride[0]} +1 \right)$$
$$W_{out} = floor\left( \frac{W_{in} + 2\times pad[1] - dilation[1] \times (kernel - 1) -1 }{stride[1]} +1 \right)$$

In *Net1*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 0``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} - kernel + 1 \right)$$
$$W_{out} = floor\left( W_{in} - kernel + 1 \right)$$

In *Net2*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 1``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} + 2 - kernel + 1 \right) = floor\left( H_{in}  - kernel + 3 \right)$$
$$W_{out} = floor\left( W_{in} + 2 - kernel + 1 \right) = floor\left( W_{in}  - kernel + 3 \right)$$

